In [1]:
import numpy as np

In [2]:
def closestTo(inputList, val):
  arr = np.asarray(inputList)
  i = (np.abs(arr - val)).argmin()
  return arr[i]

In [9]:
def closestToSimple(inputList, val):
    return inputList[np.argmin(np.abs(np.array(inputList) - val))]

In [11]:
closestTo([22,4,7,2,1.5,8,7], 1)

1.5

In [12]:
closestToSimple([22,4,7,2,1.3,8,7], 1)

1.3

In [13]:
def doMape(omni, artemis):
    return np.mean(np.abs((artemis - omni) / artemis))

In [14]:
def doMapeLong(omni, artemis):
    return (1/60) * (np.sum([abs(((a - o)/a)) for o, a in zip(omni, artemis)]))

In [31]:
omni = np.random.randint(-101, 101, 60)
artemis = np.random.randint(-101, 101, 60)

In [32]:
doMape(omni, artemis)

/var/folders/qq/kxsxv0ks5j16hqtn13mpk4sw0000gn/T/ipykernel_15170/3972059137.py:2: RuntimeWarning: divide by zero encountered in divide
  return np.mean(np.abs((artemis - omni) / artemis))


inf

In [33]:
doMapeLong(omni, artemis)

/var/folders/qq/kxsxv0ks5j16hqtn13mpk4sw0000gn/T/ipykernel_15170/2132366456.py:2: RuntimeWarning: divide by zero encountered in scalar divide
  return (1/60) * (np.sum([abs(((a - o)/a)) for o, a in zip(omni, artemis)]))


inf

In [21]:
def doRatioLong(o, a):
   return  np.average([(abs(om - ar) / abs(ar)) for om, ar in zip(o, a)])

In [22]:
def doRatioShort(o, a):
    return  np.mean(np.abs(o - a) / np.abs(a))

In [34]:
doRatioLong(omni, artemis)

/var/folders/qq/kxsxv0ks5j16hqtn13mpk4sw0000gn/T/ipykernel_15170/351219452.py:2: RuntimeWarning: divide by zero encountered in scalar divide
  return  np.average([(abs(om - ar) / abs(ar)) for om, ar in zip(o, a)])


inf

In [35]:
doRatioShort(omni, artemis)

/var/folders/qq/kxsxv0ks5j16hqtn13mpk4sw0000gn/T/ipykernel_15170/4221331949.py:2: RuntimeWarning: divide by zero encountered in divide
  return  np.mean(np.abs(o - a) / np.abs(a))


inf

In [36]:
import numpy as np
import pandas as pd
from scipy.stats import pearsonr, linregress
import os

In [78]:
# Correlation metrics
def corrMetrics(omni, artemis):
    ratio = np.mean(np.abs(omni - artemis) / np.abs(artemis))
    rmse = np.sqrt(np.mean((artemis - omni) ** 2))
    rmse_Artemis = np.sqrt(np.mean(np.where(artemis != 0, (artemis - omni) ** 2 / (artemis ** 2), np.zeros_like(artemis))))
    rmse_Omni = np.sqrt(np.mean(np.where(omni != 0, (artemis - omni) ** 2 / (omni ** 2), 0)))
    rmse_Average = np.sqrt(np.mean((artemis - omni) ** 2 / (((artemis + omni) / 2) ** 2)))

    slope, intercept, *_ = linregress(artemis, omni)

    return ratio, rmse, rmse_Artemis, rmse_Omni, rmse_Average, slope, intercept

In [79]:
def calculate_metrics(omni_slice, artemis_slice):
    corrcoef, _ = pearsonr(omni_slice, artemis_slice)
    ratio, rmse, rmseArtemis, rmseOmni, rmseAverage, slopes, ints = corrMetrics(omni_slice, artemis_slice)
    return corrcoef, ratio, rmse, rmseArtemis, rmseOmni, rmseAverage, slopes, ints

In [80]:
def find_max_value_and_index(values):
    if all(c < 0 for c in values[1:]):
        return max(values[1:], key=abs), values.index(max(values[1:], key=abs))
    else:
        return max(values[1:]), values.index(max(values[1:]))

def find_min_value_and_index(values):
    return min(values[1:]), values.index(min(values[1:]))

# Find slope closest to a given value
def closest_to_value(inputList, val):
    return inputList[np.argmin(np.abs(np.array(inputList) - val))]

In [ ]:
# Function to compute the correlation metrics.
def correlate(artemis, omni, workingDir):
    keys = ['BZ_GSE'] # Variables to loop over and their respective units
    num_windows = 1 #num_windows = len(omni)-60 # Number of 1-hour blocks in the data set

    for k in keys:
        print(f'Key {k}: Computing {num_windows} windows...')

        for n in range(num_windows): # Loop over each block
            a_start = artemis.loc[artemis['Time'] == omni['Time'].iloc[n]].index[0] # Find the start index in Artemis that matches the nth window of Omni
            a_stop = artemis.loc[artemis['Time'] == omni['Time'].iloc[n+60]].index[0] # Find the stop index in Artemis that matches the n+60 th window of Omni
            hourly_velocity = np.average(omni['Vx'][n:n+60]) # Compute the average SW velocity for the interval

            metric_names = ['Pearson', 'Ratio', 'RMSE', 'RMSE Artemis', 'RMSE Omni', 'RMSE Average', 'Slopes', 'Intercepts']
            metric_stores = {name: [] for name in metric_names}
            target_stores = {name + ' targetval': [] for name in metric_names}

            for i in range(31):
                omni_slice = omni[k][n:n+60]
                artemis_slice = artemis[k][aStart-i:aStop-i]

                for metric, store in metric_stores.items():
                    values = calculate_metrics(omni_slice, artemis_slice)
                    store.extend(values)

            pearson_max_value, pearson_max_index = find_max_value_and_index(metric_stores['Pearson'])
            metric_stores['Pearson Max'] = [pearson_max_value, pearson_max_index]

In [89]:
metric_names = ['Pearson', 'Ratio', 'RMSE', 'RMSE Artemis', 'RMSE Omni', 'RMSE Average', 'Slopes', 'Intercepts']
metric_stores = {name: [] for name in metric_names}
target_stores = {name + ' targetval': [] for name in metric_names}

for i in range(31):
    omni_slice = omni[0:10]
    artemis_slice = artemis[0+i:10+i]

    for metric, store in metric_stores.items():
        values = np.mean(omni_slice + artemis_slice)
        store.append(values)

for metric, store in metric_stores.items():
    print(metric, store)

Pearson [-95.8, -90.7, -85.4, -83.6, -90.3, -91.6, -91.1, -89.9, -92.5, -96.5, -99.3, -98.9, -103.1, -101.3, -100.0, -104.7, -103.2, -101.5, -94.8, -92.1, -87.1, -92.1, -88.3, -95.4, -93.5, -90.8, -96.5, -99.4, -104.4, -110.8, -116.9]
Ratio [-95.8, -90.7, -85.4, -83.6, -90.3, -91.6, -91.1, -89.9, -92.5, -96.5, -99.3, -98.9, -103.1, -101.3, -100.0, -104.7, -103.2, -101.5, -94.8, -92.1, -87.1, -92.1, -88.3, -95.4, -93.5, -90.8, -96.5, -99.4, -104.4, -110.8, -116.9]
RMSE [-95.8, -90.7, -85.4, -83.6, -90.3, -91.6, -91.1, -89.9, -92.5, -96.5, -99.3, -98.9, -103.1, -101.3, -100.0, -104.7, -103.2, -101.5, -94.8, -92.1, -87.1, -92.1, -88.3, -95.4, -93.5, -90.8, -96.5, -99.4, -104.4, -110.8, -116.9]
RMSE Artemis [-95.8, -90.7, -85.4, -83.6, -90.3, -91.6, -91.1, -89.9, -92.5, -96.5, -99.3, -98.9, -103.1, -101.3, -100.0, -104.7, -103.2, -101.5, -94.8, -92.1, -87.1, -92.1, -88.3, -95.4, -93.5, -90.8, -96.5, -99.4, -104.4, -110.8, -116.9]
RMSE Omni [-95.8, -90.7, -85.4, -83.6, -90.3, -91.6, -91.1, 

In [69]:
projDir ='/Volumes/Research'
artemisDirectory = os.path.join(projDir, 'Solar-Wind-Reliability/output-data/GSE/Artemis/')
omniDirectory = os.path.join(projDir, 'Solar-Wind-Reliability/output-data/GSE/Omni/')

omniFileList = sorted(filter(lambda x: os.path.isfile(os.path.join(omniDirectory, x)), os.listdir(omniDirectory)))
artemisFileList = sorted(filter(lambda x: os.path.isfile(os.path.join(artemisDirectory, x)), os.listdir(artemisDirectory)))
omniFileList.remove('.DS_Store')
artemisFileList.remove('.DS_Store')
for nums, files in enumerate(omniFileList):
    print('{} - {}'.format(nums, files))
toCorrelate = input('Please enter the number of the file you wish to correlate, or type "all" to correlate the whole folder: ')

if toCorrelate == 'all':
    for l in range(len(omniFileList)):
        if not omniFileList[l].startswith('.') and not 'copy' in omniFileList[l]:
            print('Correlating {}...'.format(omniFileList[l]))
            artemisData = pd.read_csv(os.path.join(artemisDirectory, artemisFileList[l]), delimiter=',', header=0)
            omniData = pd.read_csv(os.path.join(omniDirectory, omniFileList[l]), delimiter=',', header=0)
            artemisData['Time'] = pd.to_datetime(artemisData['Time'], format='%Y-%m-%d %H:%M:%S')
            omniData['Time'] = pd.to_datetime(omniData['Time'], format='%Y-%m-%d %H:%M:%S')

            correlate(artemisData, omniData, projDir)

elif int(toCorrelate) < len(omniFileList):
    print('Correlating {} and {}...'.format(omniFileList[int(toCorrelate)], artemisFileList[int(toCorrelate)]))
    artemisData = pd.read_csv(os.path.join(artemisDirectory, artemisFileList[int(toCorrelate)]), delimiter=',', header=0)
    omniData = pd.read_csv(os.path.join(omniDirectory, omniFileList[int(toCorrelate)]), delimiter=',', header=0)
    artemisData['Time'] = pd.to_datetime(artemisData['Time'], format='%Y-%m-%d %H:%M:%S')
    omniData['Time'] = pd.to_datetime(omniData['Time'], format='%Y-%m-%d %H:%M:%S')

    a = correlate(artemisData, omniData, projDir)

else:
    raise ValueError('Must be "any" or less than {}'.format(len(omniFileList)))


0 - Omni_2011-10-24_18-10.csv
1 - Omni_2011-12-28_16-20.csv
2 - Omni_2013-01-17_10-30.csv
3 - Omni_2014-02-27_16-30.csv
4 - Omni_2014-02-27_18-30.csv
5 - Omni_2014-04-05_19-30.csv
6 - Omni_2015-03-16_03-50.csv
7 - Omni_2015-03-17_04-20.csv
8 - Omni_2015-04-20_20-10.csv
9 - Omni_2015-06-12_17-10.csv
10 - Omni_2016-06-05_05-00.csv
11 - Omni_2016-08-02_12-15.csv
12 - Omni_2017-09-15_12-00.csv
13 - Omni_2017-09-27_04-30.csv
14 - Omni_2018-09-13_00-00.csv
15 - Omni_2018-10-09_04-00.csv
16 - Omni_2019-11-27_13-00.csv
Correlating Omni_2011-10-24_18-10.csv and Artemis_2011-10-24_18-10.csv...
Key BZ_GSE: Computing 1 windows...
60 60 90
60 60 89
60 60 88
60 60 87
60 60 86
60 60 85
60 60 84
60 60 83
60 60 82
60 60 81
60 60 80
60 60 79
60 60 78
60 60 77
60 60 76
60 60 75
60 60 74
60 60 73
60 60 72
60 60 71
60 60 70
60 60 69
60 60 68
60 60 67
60 60 66
60 60 65
60 60 64
60 60 63
60 60 62
60 60 61
60 60 60


In [59]:
a[0]

array([3.47757449, 3.47868571, 3.47009936, 3.43550103, 3.42020428,
       3.44114815, 3.45414943, 3.38367207, 3.29776515, 3.25561833,
       3.51954362, 3.50055228, 3.43016661, 3.36168471, 3.31099348,
       3.25128459, 3.19629641, 3.1467014 , 3.09359263, 3.07159669,
       3.02551717, 2.97381522, 2.91681691, 2.87503523, 2.83170403,
       2.79350525, 2.76163965, 2.72580908, 2.68712688, 2.65225202,
       2.61253998])

In [91]:
metric_names = ['Pearson', 'Ratio', 'RMSE', 'RMSE Artemis', 'RMSE Omni', 'RMSE Average', 'Slopes', 'Intercepts']
metric_stores = {name: [] for name in metric_names}

In [92]:
metric_stores

{'Pearson': [],
 'Ratio': [],
 'RMSE': [],
 'RMSE Artemis': [],
 'RMSE Omni': [],
 'RMSE Average': [],
 'Slopes': [],
 'Intercepts': []}

In [96]:
for metric, store in metric_stores.items():
    print(metric_stores[metric])

[]
[]
[]
[]
[]
[]
[]
[]
